# Agentic AI: Smart assistant
5 steps loop: 
- Get the mission: Goal
- Scan the scene: Gather needed context/information
- Think through: Plan
- Take action: Execute plan
- Learn and get better: Revise + Learn

Evolution: LLM => RAG => Agentic AI
Agentic AI: team of specialized agents working in concert to achieve complex goal

# 3 level of AI agent complexity:
- Level 0: The core reasoning engine LLM - The LLM is able to reasoned based on its own knowledge, but lacks of updated data
- Level 1: The connected problem-solver - The LLM becomes a functional agent by connecting with external tool to gather data
- Level 2: The strategic problem-solver - The LLM agent moves beyond single-tool use to tackle complex, multi-part problems
- Level 3: Collaborative AI-agents - The specialized LLM agents work together to handle a compex problem

# 5 visions of agentic AI:
- Generalist AI: AI agents will evolve from narrow specialists into true generalists capable of managing complex, ambiguous, and long-term goals with high reliability.
- Deep personalization and proactive goal discovery: AI agents will become deep personalized assistant and proactive partners
- Embodiment and Physical world interaction: AI agents is not just virtual assistant, but are embedded to the physical systems
- Agent-driven economy: Autonomous AI agents participate in the economy and replace human labours
- Goal-driven, metamorphic multi-agent system: AI agents that have the ability to analyze its own performance and modify the topology of its multi-agent workforce, creating, duplicating, or removing agents as needed to form the most effective team for the task at hand.
  
This evolution happens at multiple levels:
+ Architectural Modification: At the deepest level, individual agents can rewrite their own source code and re-architect their internal structures for higher efficiency, as in the original hypothesis.
+ Instructional Modification: At a higher level, the system continuously performs automatic prompt engineering and context engineering. It refines the instructions and information given to each agent, ensuring they are operating with optimal guidance without any human intervention.


In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Chaining
- The core idea is to break down the original, daunting problem into a sequence of smaller, more manageable sub-problems.
- Each sub-problem is addressed individually through a specifically designed prompt, and the output generated from one prompt is strategically fed as input into the subsequent prompt in the chain.


In [3]:
# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
    "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

prompt_goal = ChatPromptTemplate.from_template("Give me some laptop suggestion (model and price) with the following details:\n\n{details}")

# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()

spec_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)


full_chain = (
    {"details": spec_chain}
    | prompt_goal
    | llm
    | StrOutputParser()
    
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)


--- Final JSON Output ---
1. Dell XPS 15 - $1,799
2. MacBook Pro 16-inch - $2,399
3. ASUS ROG Zephyrus G14 - $1,449
4. HP Spectre x360 - $1,499
5. Lenovo ThinkPad X1 Carbon - $1,599


# Routing
- Routing introduces conditional logic into an agent's operational framework, enabling a shift from a fixed execution path to a model where the agent dynamically evaluates specific criteria to select from a set of possible subsequent actions. 
- This allows for more flexible and context-aware system behavior.
- Here’s a concise summary of your passage:

**Routing Pattern Core:**
The central mechanism evaluates input and directs the flow. It can be implemented via:

1. **LLM-based Routing** – Prompt the LLM to classify input and return a route identifier (flexible, but generative).
2. **Embedding-based Routing** – Compare query embeddings with route embeddings for semantic similarity (meaning-driven).
3. **Rule-based Routing** – Use explicit logic or keyword rules (fast, deterministic, but rigid).
4. **ML Model-based Routing** – Train a discriminative classifier on labeled data; routing is encoded in learned weights (specialized, supervised, distinct from LLM prompts).

👉 Together, these cover generative, semantic, deterministic, and supervised approaches to routing.





In [4]:
# --- Define Simulated Sub-Agent Handlers (equivalent to ADK sub_agents) ---
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

def booking_handler(request: str) -> str:
    """Simulates the Booking Agent handling a request."""
    print("\n--- DELEGATING TO BOOKING HANDLER ---")
    return f"Booking Handler processed request: '{request}'. Result: Simulated booking action."

def info_handler(request: str) -> str:
    """Simulates the Info Agent handling a request."""
    print("\n--- DELEGATING TO INFO HANDLER ---")
    return f"Info Handler processed request: '{request}'. Result: Simulated information retrieval."

def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    print("\n--- HANDLING UNCLEAR REQUEST ---")
    return f"Coordinator could not delegate request: '{request}'. Please clarify."

# --- Define Coordinator Router Chain (equivalent to ADK coordinator's instruction) ---
# This chain decides which handler to delegate to.
coordinator_router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Analyze the user's request and determine which specialist handler should process it.
     - If the request is related to booking flights or hotels, output 'booker'.
     - For all other general information questions, output 'info'.
     - If the request is unclear or doesn't fit either category, output 'unclear'.
     ONLY output one word: 'booker', 'info', or 'unclear'."""),
    ("user", "{user_input}")
])

llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)
coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

# Use RunnableBranch to route based on the router chain's output.

# Define the branches for the RunnableBranch
branches = {
    "booker": RunnablePassthrough.assign(output=lambda x: booking_handler(x['request']['user_input'])),
    "info": RunnablePassthrough.assign(output=lambda x: info_handler(x['request']['user_input'])),
    "unclear": RunnablePassthrough.assign(output=lambda x: unclear_handler(x['request']['user_input'])),
}

# Create the RunnableBranch. It takes the output of the router chain
# and routes the original input ('request') to the corresponding handler.
delegation_branch = RunnableBranch(
    (lambda x: x['decision'].strip() == 'booker', branches["booker"]), # Added .strip()
    (lambda x: x['decision'].strip() == 'info', branches["info"]),     # Added .strip()
    branches["unclear"] # Default branch for 'unclear' or any other output
)

# Combine the router chain and the delegation branch into a single runnable
# The router chain's output ('decision') is passed along with the original input ('request')
# to the delegation_branch.
coordinator_agent = {
    "decision": coordinator_router_chain,
    "request": RunnablePassthrough()
} | delegation_branch | (lambda x: x['output']) # Extract the final output

In [5]:
print("--- Running with a booking request ---")
request_a = "Book me a flight to London."
result_a = coordinator_agent.invoke({"user_input": request_a})
print(f"Final Result A: {result_a}")

print("\n--- Running with an info request ---")
request_b = "What is the capital of Italy?"
result_b = coordinator_agent.invoke({"user_input": request_b})
print(f"Final Result B: {result_b}")

print("\n--- Running with an unclear request ---")
request_c = "Sing me a song"
result_c = coordinator_agent.invoke({"user_input": request_c})
print(f"Final Result C: {result_c}")

--- Running with a booking request ---

--- DELEGATING TO BOOKING HANDLER ---
Final Result A: Booking Handler processed request: 'Book me a flight to London.'. Result: Simulated booking action.

--- Running with an info request ---

--- DELEGATING TO INFO HANDLER ---
Final Result B: Info Handler processed request: 'What is the capital of Italy?'. Result: Simulated information retrieval.

--- Running with an unclear request ---

--- HANDLING UNCLEAR REQUEST ---
Final Result C: Coordinator could not delegate request: 'Sing me a song'. Please clarify.


# Parallelization
Parallelization boosts agent performance by running independent tasks simultaneously instead of sequentially. Key applications include:

1. **Information Gathering** – Collect data from multiple sources at once (e.g., news, stocks, social media).
2. **Data Processing** – Apply different analyses concurrently (e.g., sentiment, keywords, categorization).
3. **Multi-API Interaction** – Query several APIs/tools in parallel (e.g., flights, hotels, events).
4. **Content Generation** – Create different content components simultaneously (e.g., email parts).
5. **Validation** – Run multiple checks at the same time (e.g., email, phone, address).
6. **Multi-Modal Processing** – Analyze different input modalities in parallel (e.g., text + image).
7. **Option Generation (A/B Testing)** – Produce multiple variations concurrently for quick comparison.

👉 **Benefit:** Faster, more comprehensive, and more responsive agents through concurrent execution of independent tasks.


In [6]:
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough
import asyncio

# --- Define Independent Chains --
# These three chains represent distinct tasks that can be executed in parallel.
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

summarize_chain: Runnable = (
   ChatPromptTemplate.from_messages([
       ("system", "Summarize the following topic concisely:"),
       ("user", "{topic}")
   ])
   | llm
   | StrOutputParser()
)

questions_chain: Runnable = (
   ChatPromptTemplate.from_messages([       ("system", "Generate three interesting questions about the following topic:"),
       ("user", "{topic}")
   ])
   | llm
   | StrOutputParser()
)

terms_chain: Runnable = (
   ChatPromptTemplate.from_messages([
       ("system", "Identify 5-10 key terms from the following topic, separated by commas:"),
       ("user", "{topic}")
   ])
   | llm
   | StrOutputParser()
)

# --- Build the Parallel + Synthesis Chain ---

# 1. Define the block of tasks to run in parallel. The results of these,
#    along with the original topic, will be fed into the next step.
map_chain = RunnableParallel(
   {
       "summary": summarize_chain,
       "questions": questions_chain,
       "key_terms": terms_chain,
       "topic": RunnablePassthrough(),  # Pass the original topic through
   }
)

# 2. Define the final synthesis prompt which will combine the parallel results.
synthesis_prompt = ChatPromptTemplate.from_messages([
   ("system", """Based on the following information:
    Summary: {summary}
    Related Questions: {questions}
    Key Terms: {key_terms}
    Synthesize a comprehensive answer."""),
   ("user", "Original topic: {topic}")
])

# 3. Construct the full chain by piping the parallel results directly
#    into the synthesis prompt, followed by the LLM and output parser.
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

# --- Run the Chain ---
async def run_parallel_example(topic: str) -> None:
   """
   Asynchronously invokes the parallel processing chain with a specific topic
   and prints the synthesized result.

   Args:
       topic: The input topic to be processed by the LangChain chains.
   """
   if not llm:
       print("LLM not initialized. Cannot run example.")
       return

   print(f"\n--- Running Parallel LangChain Example for Topic: '{topic}' ---")
   try:
       # The input to `ainvoke` is the single 'topic' string, 
       # then passed to each runnable in the `map_chain`.
       response = await full_parallel_chain.ainvoke(topic)
       print("\n--- Final Response ---")
       print(response)
   except Exception as e:
       print(f"\nAn error occurred during chain execution: {e}")

test_topic = "The history of space exploration"
# In Python 3.7+, asyncio.run is the standard way to run an async function.
await run_parallel_example(test_topic)



--- Running Parallel LangChain Example for Topic: 'The history of space exploration' ---

--- Final Response ---
The history of space exploration is a fascinating journey that has seen significant advancements in technology and human achievement. Key milestones in this history include the launch of the first artificial satellites, such as Sputnik by the Soviet Union and Explorer 1 by the United States, marking the beginning of the Space Race. This competition between the two superpowers led to the first human spaceflight by Yuri Gagarin in 1961 and culminated in the historic Apollo 11 mission in 1969, where astronauts Neil Armstrong and Buzz Aldrin became the first humans to set foot on the moon.

Following the success of the Apollo missions, space exploration continued with the launch of space probes to explore other planets in our solar system. Missions like Voyager, Cassini, and Curiosity have provided valuable insights into the geology, atmosphere, and potential for life on planet

# Reflection
Reflection introduces a feedback loop where the agent critiques and improves its own outputs. It is especially useful when quality, accuracy, or complex constraints matter.

* **Creative Writing** – Draft → critique → rewrite → repeat → produces polished content.
* **Code Generation** – Write code → test/analyze → fix → improves robustness.
* **Complex Problem Solving** – Evaluate steps → backtrack/refine → handles intricate reasoning.
* **Summarization** – Draft summary → compare to source → refine → ensures accuracy/completeness.
* **Planning** – Propose plan → evaluate feasibility → revise → creates effective strategies.
* **Conversational Agents** – Review conversation → adjust response → yields coherent, natural dialogue.

👉 **Benefit:** Reflection acts as meta-cognition, letting agents learn from and refine their outputs, leading to more reliable, intelligent, and high-quality results.


In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

# Initialize the Chat LLM. We use gpt-4o for better reasoning.
# A lower temperature is used for more deterministic outputs.
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

def run_reflection_loop():
   """
   Demonstrates a multi-step AI reflection loop to progressively improve a Python function.
   """
   # --- The Core Task ---
   task_prompt = """
   Your task is to create a Python function named `calculate_factorial`.
   This function should do the following:
   1.  Accept a single integer `n` as input.
   2.  Calculate its factorial (n!).
   3.  Include a clear docstring explaining what the function does.
   4.  Handle edge cases: The factorial of 0 is 1.
   5.  Handle invalid input: Raise a ValueError if the input is a negative number.
   """
   # --- The Reflection Loop ---
   max_iterations = 3
   current_code = ""
   # We will build a conversation history to provide context in each step.
   message_history = [HumanMessage(content=task_prompt)]

   for i in range(max_iterations):
       print("\n" + "="*25 + f" REFLECTION LOOP: ITERATION {i + 1} " + "="*25)

       # --- 1. GENERATE / REFINE STAGE ---
       # In the first iteration, it generates. In subsequent iterations, it refines.
       if i == 0:
           print("\n>>> STAGE 1: GENERATING initial code...")
           # The first message is just the task prompt.
           response = llm.invoke(message_history)
           current_code = response.content
       else:
           print("\n>>> STAGE 1: REFINING code based on previous critique...")
           # The message history now contains the task, 
           # the last code, and the last critique.
           # We instruct the model to apply the critiques.
           message_history.append(HumanMessage(content="Please refine the code using the critiques provided."))
           response = llm.invoke(message_history)
           current_code = response.content

       print("\n--- Generated Code (v" + str(i + 1) + ") ---\n" + current_code)
       message_history.append(response) # Add the generated code to history

       # --- 2. REFLECT STAGE ---
       print("\n>>> STAGE 2: REFLECTING on the generated code...")

       # Create a specific prompt for the reflector agent.
       # This asks the model to act as a senior code reviewer.
       reflector_prompt = [
           SystemMessage(content="""
               You are a senior software engineer and an expert 
               in Python.
               Your role is to perform a meticulous code review.
               Critically evaluate the provided Python code based 
               on the original task requirements.
               Look for bugs, style issues, missing edge cases, 
               and areas for improvement.
               If the code is perfect and meets all requirements,
               respond with the single phrase 'CODE_IS_PERFECT'.
               Otherwise, provide a bulleted list of your critiques.
           """),
           HumanMessage(content=f"Original Task:\n{task_prompt}\n\nCode to Review:\n{current_code}")
       ]

       critique_response = llm.invoke(reflector_prompt)
       critique = critique_response.content

       # --- 3. STOPPING CONDITION ---
       if "CODE_IS_PERFECT" in critique:
           print("\n--- Critique ---\nNo further critiques found. The code is satisfactory.")
           break

       print("\n--- Critique ---\n" + critique)
       # Add the critique to the history for the next refinement loop.
       message_history.append(HumanMessage(content=f"Critique of the previous code:\n{critique}"))

   print("\n" + "="*30 + " FINAL RESULT " + "="*30)
   print("\nFinal refined code after the reflection process:\n")
   print(current_code)

run_reflection_loop()



========================= REFLECTION LOOP: ITERATION 1 =========================

>>> STAGE 1: GENERATING initial code...

--- Generated Code (v1) ---
Here's a Python function named `calculate_factorial` that meets the specified requirements:

```python
def calculate_factorial(n):
    """
    Calculate the factorial of a non-negative integer n.

    The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.
    The factorial of 0 is defined as 1.

    Parameters:
    n (int): A non-negative integer whose factorial is to be calculated.

    Returns:
    int: The factorial of the input integer n.

    Raises:
    ValueError: If the input is a negative integer.
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    
    factorial = 1
    for i in range(2, n + 1):
        factorial *= i
    
    return factorial

# Example usage:
# print(calculate_factorial(5))  # Output: 120
# print(calculate_fa

In [8]:
#Rewrite the code in LangChain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

TASK_PROMPT = """
Your task is to create a Python function named `calculate_factorial`.
This function should:
1) Accept a single integer `n` as input.
2) Calculate its factorial (n!).
3) Include a clear docstring.
4) Handle edge case: factorial(0) == 1.
5) Handle invalid input: raise ValueError if n < 0.
"""

# ---------- Writer (generate/refine) ----------
writer_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a senior Python developer. You will write or refine code based on a task and (optionally) critique."),
    ("human",
     "TASK:\n{task}\n\n"
     "PREVIOUS_CODE (may be empty):\n{previous_code}\n\n"
     "CRITIQUE (may be empty):\n{critique}\n\n"
     "Write the best possible complete Python code now.")
])
writer_chain = writer_prompt | llm | parser

# ---------- Reviewer (reflection) ----------
reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a meticulous Python code reviewer. "
     "Evaluate against the task. If the code fully meets requirements, reply EXACTLY: CODE_IS_PERFECT. "
     "Else, return a concise bullet list of issues and improvements."),
    ("human",
     "TASK:\n{task}\n\nCODE TO REVIEW:\n{code}")
])
reviewer_chain = reviewer_prompt | llm | parser

# ---------- Reflection loop ----------
def run_reflection_loop(max_iterations: int = 3):
    current_code = ""
    critique = ""

    for i in range(max_iterations):
        print("\n" + "="*25 + f" REFLECTION ITERATION {i+1} " + "="*25)

        # 1) Generate / Refine
        print("\n>>> STAGE 1: GENERATE/REFINE")
        current_code = writer_chain.invoke({
            "task": TASK_PROMPT,
            "previous_code": current_code,
            "critique": critique
        })
        print("\n--- Generated Code ---\n", current_code)

        # 2) Reflect
        print("\n>>> STAGE 2: REFLECT")
        critique = reviewer_chain.invoke({
            "task": TASK_PROMPT,
            "code": current_code
        })
        if "CODE_IS_PERFECT" in critique:
            print("\n--- Critique ---\nNo further critiques. CODE_IS_PERFECT")
            break

        print("\n--- Critique ---\n", critique)

    print("\n" + "="*30 + " FINAL RESULT " + "="*30)
    print("\nFinal refined code:\n")
    print(current_code)

# Run
run_reflection_loop()




========================= REFLECTION ITERATION 1 =========================

>>> STAGE 1: GENERATE/REFINE

--- Generated Code ---
 Here is the complete Python code for the `calculate_factorial` function, which meets all the specified requirements:

```python
def calculate_factorial(n: int) -> int:
    """
    Calculate the factorial of a non-negative integer n.

    The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.
    It is denoted as n! and defined as:
    - n! = n × (n - 1) × (n - 2) × ... × 1 for n > 0
    - 0! = 1

    Parameters:
    n (int): A non-negative integer for which to calculate the factorial.

    Returns:
    int: The factorial of the given integer n.

    Raises:
    ValueError: If n is a negative integer.
    """
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")
    elif n == 0:
        return 1
    else:
        factorial = 1
        for i in range(1, n + 1):
            factorial

# Tool Use
Tool use enables agents to move beyond text generation, letting them act, query, and interact with external systems.

* **Information Retrieval** – Call APIs (e.g., weather) → fetch real-time data → give user-friendly answers.
* **Databases & APIs** – Query/update structured data (e.g., inventory, orders) → deliver accurate status.
* **Calculations & Analysis** – Use calculators, data APIs, or libraries → perform numeric/financial reasoning.
* **Communications** – Trigger email, messaging, or notifications → act as a personal assistant.
* **Code Execution** – Run snippets via interpreters → analyze and explain program behavior.
* **System Control** – Interact with devices/IoT (e.g., smart lights) → take real-world actions.

👉 **Benefit:** Tool use transforms an LLM from a text-only model into a true **agent** that can sense, reason, and act in digital or physical environments.


In [9]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [10]:
import asyncio
import re
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

# --- LLM & parser ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# --- Define a Tool ---
def _normalize(s: str) -> str:
    # lowercase, strip punctuation & extra spaces
    s = s.lower()
    s = re.sub(r"[^\w\s]", "", s)
    return re.sub(r"\s+", " ", s).strip()

@tool
def search_information(query: str) -> str:
    """
    Provides factual information on a given topic. Use this tool to find answers to questions
    like 'What is the capital of France?' or 'What is the weather in London?'.
    """
    print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
    normalized = _normalize(query)

    simulated_results = {
        "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
        "capital of france": "The capital of France is Paris.",
        "population of earth": "The estimated population of Earth is around 8 billion people.",
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
    }
    result = simulated_results.get(normalized, f"Simulated search result for '{query}': No specific information found, but the topic seems interesting.")
    print(f"--- TOOL RESULT: {result} ---")
    return result

tools = [search_information]

# --- Prompt for tool-calling agent ---
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. If a tool is relevant, call it; otherwise answer directly."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),  # required for tool use traces
])

# --- Create agent and executor ---
agent = create_tool_calling_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Async runner helpers ---
async def run_agent_with_tool(query: str):
    print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
    try:
        response = await agent_executor.ainvoke({"input": query})
        print("\n--- ✅ Final Agent Response ---")
        print(response["output"])
    except Exception as e:
        print(f"\n🛑 An error occurred during agent execution: {e}")

async def agent_with_tool():
    tasks = [
        run_agent_with_tool("What is the capital of France?"),
        run_agent_with_tool("What's the weather like in London?"),
        run_agent_with_tool("Tell me something about dogs."),  # default branch
    ]
    await asyncio.gather(*tasks)


In [11]:
await agent_with_tool()


--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'dogs'}`



--- 🛠️ Tool Called: search_information with query: 'dogs' ---
--- TOOL RESULT: Simulated search result for 'dogs': No specific information found, but the topic seems interesting. ---
Simulated search result for 'dogs': No specific information found, but the topic seems interesting.
Invoking: `search_information` with `{'query': 'current weather in London'}`



--- 🛠️ Tool Called: search_information with query: 'current weather in London' ---
--- TOOL RESULT: Simulated search result for 'current weather in London': No specific information found, but the topic seems interesting. ---
Simulated search re

# Planning
Planning enables agents to go beyond reactive responses by formulating a **sequence of actions** that lead from an initial state to a goal state.

* **Core Idea** – The agent discovers the *how*, not just executes the *what*. It breaks down complex goals into manageable steps, adapts when constraints change, and replans when obstacles arise.
* **Trade-off** – Use planning when the solution path is uncertain or dynamic. For repeatable, well-defined tasks, fixed workflows are more predictable and reliable.
* **Applications**:

  * **Task automation** – orchestrating multi-step business processes (e.g., employee onboarding).
  * **Robotics/navigation** – generating paths under constraints (e.g., obstacle avoidance, efficiency).
  * **Information synthesis** – structuring outputs like research reports in phases.
  * **Customer support** – diagnosing and resolving multi-step problems systematically.

👉 **Benefit:** Planning equips agents with foresight, adaptability, and goal-oriented reasoning, making them effective in complex, uncertain, or evolving environments.


In [14]:
from crewai import Agent, Task, Crew, Process

In [15]:
# 2. Define a clear and focused agent
planner_writer_agent = Agent(
    role='Article Planner and Writer',
    goal='Plan and then write a concise, engaging summary on a specified topic.',
    backstory=(
        'You are an expert technical writer and content strategist. '
        'Your strength lies in creating a clear, actionable plan before writing, '
        'ensuring the final summary is both informative and easy to digest.'
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm # Assign the specific LLM to the agent
)

# 3. Define a task with a more structured and specific expected output
topic = "The importance of Reinforcement Learning in AI"
high_level_task = Task(
    description=(
        f"1. Create a bullet-point plan for a summary on the topic: '{topic}'.\n"
        f"2. Write the summary based on your plan, keeping it around 200 words."
    ),
    expected_output=(
        "A final report containing two distinct sections:\n\n"
        "### Plan\n"
        "- A bulleted list outlining the main points of the summary.\n\n"
        "### Summary\n"
        "- A concise and well-structured summary of the topic."
    ),
    agent=planner_writer_agent,
)

# Create the crew with a clear process
crew = Crew(
    agents=[planner_writer_agent],
    tasks=[high_level_task],
    process=Process.sequential,
)

# Execute the task
print("## Running the planning and writing task ##")
result = crew.kickoff()

# print("\n\n---\n## Task Result ##\n---")
# print(result)

## Running the planning and writing task ##


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Article Planner and Writer                                                                              │
│                                                                                                                 │
│  Task: 1. Create a bullet-point plan for a summary on the topic: 'The importance of Reinforcement Learning in   │
│  AI'.                                                                                                           │
│  2. Write the summary based on your plan, keeping it around 200 words.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Article Planner and Writer                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Plan                                                                                                       │
│  - Define Reinforcement Learning (RL) and its role in AI.                                                       │
│  - Explain the key principles of RL: agents, environments, actions, rewards.                                    │
│  - Discuss the significance of RL in decision-making processes.                                                 │
│  - Highlight real-world applications of RL in various industries (e.g., gaming, robotics, healthcare).          │
│  - Mention the advantages of RL over traditional machine learning methods.                                      │
│  - Conclude with the future potential of RL in advancing AI technologies.                                       │
│                                                                                                                 │
│  ### Summary                                                                                                    │
│  Reinforcement Learning (RL) is a crucial subset of artificial intelligence that focuses on how agents should   │
│  take actions in an environment to maximize cumulative rewards. The core principles of RL involve agents        │
│  interacting with their environment, making decisions, and receiving feedback in the form of rewards or         │
│  penalties. This framework is particularly significant in decision-making processes, where RL enables systems   │
│  to learn optimal strategies through trial and error.                                                           │
│                                                                                                                 │
│  Real-world applications of RL are diverse, spanning industries such as gaming, where it powers advanced AI     │
│  opponents, to robotics, where it helps machines learn complex tasks. In healthcare, RL is being utilized to    │
│  optimize treatment plans and improve patient outcomes.                                                         │
│                                                                                                                 │
│  One of the key advantages of RL over traditional machine learning methods is its ability to adapt and improve  │
│  through experience, making it highly effective in dynamic environments. As research and development in RL      │
│  continue to advance, its potential to revolutionize AI technologies and applications is immense, paving the    │
│  way for smarter, more autonomous systems.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Multi-Agent Collaboration Models
Designing effective multi-agent systems requires careful choice of interrelationships and communication structures, each with unique strengths and trade-offs:

* **Single Agent** – Independent, simple, but limited in scope.
* **Network** – Peer-to-peer collaboration, resilient but prone to communication overhead.
* **Supervisor** – Centralized control and coordination, simplifies management but risks bottlenecks and single points of failure.
* **Supervisor as a Tool** – Supervisor provides resources/guidance without strict control, balancing support and autonomy.
* **Hierarchical** – Multi-layered supervision for complex, decomposable tasks; scalable but rigid.
* **Custom** – Tailored or hybrid structures optimized for specific goals, dynamic environments, or domain needs.

👉 **Key takeaway:** No single model is best; the optimal choice depends on task complexity, agent count, autonomy needs, robustness requirements, and communication overhead. Future systems will likely blend and extend these models to achieve more adaptive collaborative intelligence.


In [16]:
# pip install crewai langchain-openai

from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# --- LLM (crew-level) ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Agents ---
researcher = Agent(
    role='Senior Research Analyst',
    goal='Find and summarize the latest trends in applying Small Language Models for Agentic AI.',
    backstory="You identify key trends and synthesize findings.",
    verbose=True,
    allow_delegation=False,
    # llm=llm,  # optional when provided at Crew level
)

writer = Agent(
    role='Technical Content Writer',
    goal='Write a clear and engaging blog post based on research findings.',
    backstory="You translate complex topics into accessible content.",
    verbose=True,
    allow_delegation=False,
    # llm=llm,
)

# --- Tasks ---
research_task = Task(
    description=(
        "Research the top 3 emerging trends in applying Small Language Models for Agentic AI. "
        "Provide bullet points with brief explanations and (if known) notable papers."
    ),
    expected_output=(
        "A structured summary: 3 trends with 2–3 bullets each; include short rationale and any sources."
    ),
    agent=researcher,
)

writing_task = Task(
    description=(
        "Write a ~500-word blog post for a general audience based on the research summary. "
        "Hook -> overview -> 3 trend sections -> closing takeaway."
    ),
    expected_output="A polished ~500-word post in markdown.",
    agent=writer,
    context=[research_task],  # uses the researcher’s output
)

# --- Crew ---
blog_creation_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,  # researcher -> writer
    llm=llm,
    verbose=True,
)

# --- Run ---
print("## Running the blog creation crew with OpenAI gpt-4o-mini... ##")
try:
    result = blog_creation_crew.kickoff()
    print("\n------------------\n## Crew Final Output ##\n")
    print(result)
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")


## Running the blog creation crew with OpenAI gpt-4o-mini... ##


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d1a8d4ed-b643-44e4-b29e-130993bb6879                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Research the top 3 emerging trends in applying Small Language Models for Agentic AI. Provide bullet      │
│  points with brief explanations and (if known) notable papers.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Emerging Trends in Applying Small Language Models for Agentic AI                                           │
│                                                                                                                 │
│  1. **Task-Specific Fine-Tuning**                                                                               │
│     - Small language models are increasingly being fine-tuned for specific tasks, enhancing their performance   │
│  in domains such as healthcare, finance, and customer service. This approach allows agents to perform narrowly  │
│  defined responsibilities while maintaining a lightweight structure, optimizing resource use without            │
│  compromising output quality.                                                                                   │
│     - *Notable Paper:* "Fine-Tuning Small Language Models for Enhanced Task Performance" (2022) highlights      │
│  methods of fine-tuning smaller architectures to replicate the capabilities of larger models in niche           │
│  applications.                                                                                                  │
│                                                                                                                 │
│  2. **Energy Efficiency and Low-Resource Deployment**                                                           │
│     - There is a significant trend toward developing smaller models that can run efficiently on low-resource    │
│  devices, such as mobile phones and IoT devices. This trend is driven by the need for portable, real-time       │
│  processing capabilities, enabling AI agents to function independently in various environments while            │
│  minimizing power consumption.                                                                                  │
│     - *Notable Paper:* "Low-Resource Language Models for Edge Computing" (2023) discusses the advantages of     │
│  deploying smaller models across decentralized systems, underscoring the importance of energy-efficient AI in   │
│  everyday applications.                                                                                         │
│                                                                                                                 │
│  3. **Explainability and Ethical Considerations**                                                               │
│     - The integration of ethical guidelines and explainability features within small language models becomes    │
│  paramount as these agents take on more decision-making roles. Researchers focus on enhancing transparency and  │
│  interpretability to ensure that AI-driven decisions can be understood and trusted by users. This trend         │
│  responds to increasing concerns over AI accountability and fairness in automated systems.                      │
│     - *Notable Paper:* "Explainability in Small Language Models for Agentic Systems" (2023) emphasizes          │
│  strategies for incorporating interpretability into smaller models, thus ensuring that agentic AI aligns with   │
│  human values and ethical standards.                                                                            │
│                                                                                                                 │
│  Overall, these trends showcase the potential of small 

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Content Writer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Emerging Trends in Applying Small Language Models for Agentic AI                                             │
│                                                                                                                 │
│  Artificial intelligence is omnipresent in our daily lives, from chatbots assisting us with customer service    │
│  to advanced algorithms that contribute to critical decisions in healthcare. Recent research highlights an      │
│  exciting development in AI technology—the emergence of small language models specifically designed for         │
│  agentic AI. These models are gaining ground not only for their efficiency but also for their ethical           │
│  applicability in various fields. Here are three key trends in this burgeoning domain.                          │
│                                                                                                                 │
│  ## 1. Task-Specific Fine-Tuning                                                                                │
│                                                                                                                 │
│  The application of small language models is evolving as they are increasingly customized through               │
│  task-specific fine-tuning. This involves training these models on particular datasets related to sectors like  │
│  healthcare, finance, and customer service, enhancing their accuracy and performance. For instance, a small     │
│  language model fine-tuned for medical diagnosis can efficiently analyze patterns in patient data, ultimately   │
│  improving patient outcomes without requiring the hefty computational resources demanded by larger models.      │
│                                                                                                                 │
│  A notable paper titled *"Fine-Tuning Small Language Models for Enhanced Task Performance"* (2022) illustrates  │
│  unique methods of adapting small architectures to rival the capabilities of their larger counterparts for      │
│  specialized applications. This trend optimizes the balance between processing power and output quality,        │
│  ensuring these AI agents can assume narrowly defined responsibilities effectively.                             │
│                                                                                                                 │
│  ## 2. Energy Efficiency and Low-Resource Deployment                                                            │
│                                                                                                                 │
│  As the demand for portable technology continues to grow, there's a significant focus on the energy efficiency  │
│  of small language models. These models can run on low-resource devices—like mobile phones and Internet of      │
│  Things (IoT) devices—by being lightweight yet powerful enough to process information in real-time.             │
│                                                                                                                 │
│  The paper *"Low-Resource Language Models for Edge Computing"* (2023) discusses the substantial advantages of   │
│  deploying smaller models across decentralized systems.

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3415c76d-2106-45a4-83ef-5565392dc9f7                                                                     │
│  Agent: Technical Content Writer                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d1a8d4ed-b643-44e4-b29e-130993bb6879                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Emerging Trends in Applying Small Language Models for Agentic AI                                             │
│                                                                                                                 │
│  Artificial intelligence is omnipresent in our daily lives, from chatbots assisting us with customer service    │
│  to advanced algorithms that contribute to critical decisions in healthcare. Recent research highlights an      │
│  exciting development in AI technology—the emergence of small language models specifically designed for         │
│  agentic AI. These models are gaining ground not only for their efficiency but also for their ethical           │
│  applicability in various fields. Here are three key trends in this burgeoning domain.                          │
│                                                                                                                 │
│  ## 1. Task-Specific Fine-Tuning                                                                                │
│                                                                                                                 │
│  The application of small language models is evolving as they are increasingly customized through               │
│  task-specific fine-tuning. This involves training these models on particular datasets related to sectors like  │
│  healthcare, finance, and customer service, enhancing their accuracy and performance. For instance, a small     │
│  language model fine-tuned for medical diagnosis can efficiently analyze patterns in patient data, ultimately   │
│  improving patient outcomes without requiring the hefty computational resources demanded by larger models.      │
│                                                                                                                 │
│  A notable paper titled *"Fine-Tuning Small Language Models for Enhanced Task Performance"* (2022) illustrates  │
│  unique methods of adapting small architectures to rival the capabilities of their larger counterparts for      │
│  specialized applications. This trend optimizes the balance between processing power and output quality,        │
│  ensuring these AI agents can assume narrowly defined responsibilities effectively.                             │
│                                                                                                                 │
│  ## 2. Energy Efficiency and Low-Resource Deployment                                                            │
│                                                                                                                 │
│  As the demand for portable technology continues to grow, there's a significant focus on the energy efficiency  │
│  of small language models. These models can run on low-resource devices—like mobile phones and Internet of      │
│  Things (IoT) devices—by being lightweight yet powerful enough to process information in real-time.             │
│                                                                                                                 │
│  The paper *"Low-Resource Language Models for Edge Com


------------------
## Crew Final Output ##

```markdown
# Emerging Trends in Applying Small Language Models for Agentic AI

Artificial intelligence is omnipresent in our daily lives, from chatbots assisting us with customer service to advanced algorithms that contribute to critical decisions in healthcare. Recent research highlights an exciting development in AI technology—the emergence of small language models specifically designed for agentic AI. These models are gaining ground not only for their efficiency but also for their ethical applicability in various fields. Here are three key trends in this burgeoning domain.

## 1. Task-Specific Fine-Tuning

The application of small language models is evolving as they are increasingly customized through task-specific fine-tuning. This involves training these models on particular datasets related to sectors like healthcare, finance, and customer service, enhancing their accuracy and performance. For instance, a small language model fine-tun